In [37]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import datetime as dt
import random

import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
from folium.plugins import MarkerCluster

In [38]:
df = pd.read_csv("crime.csv")

In [39]:
df.dropna(subset = ["Lon"], inplace=True)
df.dropna(subset = ["Lat"], inplace=True)
df['Dispatch_Date'] = pd.to_datetime(df['Dispatch_Date'], errors='coerce')
include = df[(df.Dispatch_Date.dt.year == 2016) & (df.Dispatch_Date.dt.month == 5)]

In [40]:
#location=include.groupby(['Lat','Lon','Text_General_Code']).size().reset_index(name='cnt').sort_values(by='cnt',ascending=False)
location=include.groupby(['Lat','Lon','Text_General_Code','Dispatch_Date_Time']).size().reset_index(name='cnt').sort_values(by='cnt',ascending=False)
location.sort_values('cnt')
location.head()

,Lat,Lon,Text_General_Code,Dispatch_Date_Time,cnt
1130,39.929130,-75.187370,All Other Offenses,2016-05-28 21:55:00,6
12551,40.042713,-75.076549,All Other Offenses,2016-05-30 23:21:00,6
13661,40.058137,-75.059722,All Other Offenses,2016-05-06 22:23:00,6
10307,40.022416,-75.078182,All Other Offenses,2016-05-20 23:40:00,5
6796,39.988728,-75.187106,DRIVING UNDER THE INFLUENCE,2016-05-20 21:54:00,5


In [41]:

location['color']=location['cnt'].apply(lambda count:"black" if count==53 else
                                        "grey" if count==42 else
                                        "red" if count==31 else
                                        "violet" if count==23 else
                                        "indianred" if count==21 else
                                        "red" if count==20 else
                                        "deeppink" if count==17 else
                                        "purple" if count==16 else
                                        "darkviolet" if count==15 else
                                        "slateblue" if count==14 else
                                        "green" if count==13 else
                                        "lightgreen" if count==12 else
                                        "aqua" if count==11 else
                                        "dodgerblue" if count==10 else
                                        "darkorange" if count==9 else
                                        "orange" if count==8 else
                                        "forestgreen" if count==7 else
                                        "limegreen" if count==6 else
                                        "rawgreen" if count==5 else
                                        "darkcyan" if count==4 else
                                        "blue" if count==3 else
                                        "red" if count==2 else
                                         "green")

location['size']=location['cnt'].apply(lambda count:22 if count==53 else
                                        21 if count==42 else
                                        19 if count==31 else
                                        17 if count==23 else
                                        16 if count==21 else
                                        16 if count==20 else
                                        14 if count==17 else
                                        13 if count==16 else
                                        12 if count==15 else
                                        12 if count==14 else
                                        10 if count==13 else
                                        10 if count==12 else
                                        10 if count==11 else
                                        9 if count==10 else
                                        9 if count==9 else
                                        9 if count==8 else
                                        8 if count==7 else
                                        6 if count==6 else
                                        5 if count==5 else
                                        4 if count==4 else
                                        3 if count==3 else
                                        3 if count==2 else
                                        0.7)



In [47]:
m=folium.Map([39.9527237,-75.1635262],zoom_start=11)
for lat,lon,code,color,count,size in zip(location['Lat'],location['Lon'],location['Text_General_Code'],location['color'],location['cnt'],location['size']):
    folium.CircleMarker([lat, lon],
                            popup="Type"+code,
                            radius=size,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                            fill_color=color,
                           ).add_to(m)

m.save('PHL_map.html')

In [48]:
location_data = location[['Lat', 'Lon']].values

# plot heatmap
m.add_child(plugins.HeatMap(location_data, radius=15))
m.save('PHL_heatmap.html')

In [50]:
location_df=location

some_map_2 = folium.Map(location=[location_df['Lat'].mean(),location_df['Lon'].mean()],zoom_start=10)

mc = MarkerCluster()

for row in location_df.itertuples():
    my_string = "Type : "+row.Text_General_Code
    mc.add_child(folium.Marker(location=[row.Lat,row.Lon],popup=my_string))

some_map_2.add_child(mc)
some_map_2.save('PHLMasterCluster.html')